In [4]:
#joint test
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  4 21:31:36 2019

@author: IVPL-D11
"""

# -*- coding: utf-8 -*-
"""
Created on Wed May 15 05:52:59 2019

@author: IVPL-D11
"""
import csv
import cv2
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
from acquisitionKinect import AcquisitionKinect
from frame import Frame as Frame
import numpy as np
from scipy import io
import math
D = 1500
def cosAngle (u, v):
    cosAngle = np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))
    return cosAngle

def binarizeHV (v):
    threshold = 0
    for i in range(len (v)):
        if v [i] > threshold:
            v [i] = 1
        else:
            v [i] = -1
    return v


def discretize(data, bins):
    split = np.array_split(np.sort(data), bins)
    cutoffs = [x[-1] for x in split]
    cutoffs = cutoffs[:-1]
    discrete = np.digitize(data, cutoffs, right=True)
    return discrete

def hdcResult(AM,M0,CiM,iMjoints,iMaxis,MAXL,compareAngle):
    TP = np.zeros((1,17))
    FN = np.zeros((1,17))
    FP = np.zeros((1,17))
    predictedLabel = -1
    precision = 0
    recall = 0
    print("in")
    for j in range (3):
        for i in range(1, 4):
            if(i == 1 or i == 2 or i == 3 or i == 4 or i == 5 or i == 6 or i == 9 or i == 10 or i == 11 or i == 12 or i == 16):
                tmp = np.zeros(D)
                M = discretize(M0, MAXL)
                M = M.reshape((50, 42))
                for k in range( len(M)):
                    
                    for n in range(len(M[0])):
                        
                        jointHDC = iMjoints[math.floor(n/3)]
    
                        axisHDC = iMaxis[n%3]
                        
                        valueHDC = CiM[M[k][n]]
                       #
                        tmp += ((jointHDC*axisHDC) * valueHDC)
                    maxAngle = compareAngle
                    for b in range (1,4):
                        
                        if(b == 1 or b == 2 or b == 3 or b == 4 or b == 5 or b == 6 or b == 9 or b == 10 or b == 11 or b == 12 or b == 16):
                            angle = cosAngle (AM[b-1], binarizeHV(tmp))
                            
                            if (angle > maxAngle):
                                maxAngle = angle
                                predictedLabel = b
                    #if predictedLabel == i:
                        #TP(:,i) = TP(:,i)+1
                    #else:
                        #FN(:,i) = FN(:,i)+1;
                        #FP(:,predictedLabel) = FP(:,predictedLabel)+1
   #print("predictLabel:", predictedLabel)

    #precision = TP./(TP+FP)
    #recall = TP./(TP+FN)
    return predictedLabel


if __name__ == '__main__':   
    numActivities = 17
    MAXL = 30
    compareAngle = 0.5
    activityName = 'still'
    feature = np.zeros((20,3))
    feature_d = []
    Kinect = AcquisitionKinect()
    frame = Frame()
    i = 1
    k = 0
    #csvfile = open("C:\\Users\\IVPL-D11\\models\\research\\object_detection\\0512150222.csv", "w", newline= "")
    #csvwriter = csv.writer(csvfile)
    
    frameset = np.array([])
    norm_frameset = np.array([])
    filename = 'D1500.mat'
    AM = io.loadmat(filename, mat_dtype = False)
    
    CiM_o = AM['Cim_out']
    
    CiM = np.array([])
    for i in range(30):
        CiM = np.concatenate((CiM,np.array(CiM_o)['value'][0,0][0][int(i)][0]))
    CiM = CiM.reshape((30, D))
    
    iMaxis_o = AM['IMaxis_out']
    iMaxis = np.array([])
    for i in range(3):
        iMaxis = np.concatenate((iMaxis,np.array(iMaxis_o)['value'][0,0][0][int(i)][0]))
    iMaxis = iMaxis.reshape((3, D))
    
    iMjoints_o = AM['IMjoints_out']
    iMjoints = np.array([])
    for i in range(14):
        iMjoints = np.concatenate((iMjoints,np.array(iMjoints_o)['value'][0,0][0][int(i)][0]))
    iMjoints = iMjoints.reshape((14, D))
    
    AM_o = AM['AM2str']
    AMM = np.array([])
    for i in range(3):
        AMM = np.concatenate((AMM,np.array(AM_o)['value'][0,0][0][int(i)][0]))
    AMM = AMM.reshape((3, D))

    #key = np.array(AMM)['key'][0, 0][0][0]
    while True:
        Kinect.get_frame(frame)
        Kinect.get_color_frame()
        image = Kinect._frameRGB
        
        Kinect.get_depth_frame()
        depthOriginal = Kinect._frameDepth
        Skeleton_img = Kinect._frameSkeleton

        
        if Skeleton_img is not None:

            
            cv2.resize(Skeleton_img, (640, 480))
            cv2.imshow('body', Skeleton_img)
            
            
           
            feature[1][0] = Kinect.joint_points3D[3].x
            feature[1][1] = Kinect.joint_points3D[3].y
            feature[1][2] = Kinect.joint_points3D[3].z
            
            feature[2][0] = Kinect.joint_points3D[2].x
            feature[2][1] = Kinect.joint_points3D[2].y
            feature[2][2] = Kinect.joint_points3D[2].z
            
            feature[3][0] = Kinect.joint_points3D[1].x
            feature[3][1] = Kinect.joint_points3D[1].y
            feature[3][2] = Kinect.joint_points3D[1].z

            feature[4][0] = Kinect.joint_points3D[8].x
            feature[4][1] = Kinect.joint_points3D[8].y
            feature[4][2] = Kinect.joint_points3D[8].z

            feature[5][0] = Kinect.joint_points3D[9].x
            feature[5][1] = Kinect.joint_points3D[9].y
            feature[5][2] = Kinect.joint_points3D[9].z

            feature[6][0] = Kinect.joint_points3D[4].x
            feature[6][1] = Kinect.joint_points3D[4].y
            feature[6][2] = Kinect.joint_points3D[4].z

            feature[7][0] = Kinect.joint_points3D[5].x
            feature[7][1] = Kinect.joint_points3D[5].y
            feature[7][2] = Kinect.joint_points3D[5].z

            feature[8][0] = Kinect.joint_points3D[16].x
            feature[8][1] = Kinect.joint_points3D[16].y
            feature[8][2] = Kinect.joint_points3D[16].z

            feature[9][0] = Kinect.joint_points3D[17].x
            feature[9][1] = Kinect.joint_points3D[17].y
            feature[9][2] = Kinect.joint_points3D[17].z

            feature[10][0] = Kinect.joint_points3D[12].x
            feature[10][1] = Kinect.joint_points3D[12].y
            feature[10][2] = Kinect.joint_points3D[12].z

            feature[11][0] = Kinect.joint_points3D[13].x
            feature[11][1] = Kinect.joint_points3D[13].y
            feature[11][2] = Kinect.joint_points3D[13].z

            feature[12][0] = Kinect.joint_points3D[11].x
            feature[12][1] = Kinect.joint_points3D[11].y
            feature[12][2] = Kinect.joint_points3D[11].z

            feature[13][0] = Kinect.joint_points3D[7].x
            feature[13][1] = Kinect.joint_points3D[7].y
            feature[13][2] = Kinect.joint_points3D[7].z

            feature[14][0] = Kinect.joint_points3D[19].x
            feature[14][1] = Kinect.joint_points3D[19].y
            feature[14][2] = Kinect.joint_points3D[19].z

            feature[15][0] = Kinect.joint_points3D[15].x
            feature[15][1] = Kinect.joint_points3D[15].y
            feature[15][2] = Kinect.joint_points3D[15].z
            tmp = []
            for i in range (1, 16):
                for j in range(3):
                    tmp.append(feature[i][j])
         #   csvwriter.writerow(tmp)
         #   if k== 1102:
         #       break
            frameset = np.concatenate((frameset, tmp), axis = 0)
            k+=1
            if(k == 50):
                frameset = frameset.reshape((k, 45))
                for row in range(len(frameset)):
                    torsox = float(frameset[row][6])
                    torsoy = float(frameset[row][7])
                    torsoz = float(frameset[row][8])
                    neckx = float(frameset[row][3])
                    necky = float(frameset[row][4])
                    neckz = float(frameset[row][5])
                    denom = math.sqrt(math.pow((neckx - torsox),2) + math.pow((necky - torsoy),2) + math.pow((neckz - torsoz),2))
                    tmp = []
                    i = 0
                    while (i < 6):
                        tmp.append((float(frameset[row][i]) - torsox)/denom)
                        tmp.append((float(frameset[row][i+1]) - torsoy)/denom)
                        tmp.append((float(frameset[row][i+2]) - torsoz)/denom)
                        i+=3
                    i = 9
                    while (i < 45):
                        tmp.append((float(frameset[row][i]) - torsox)/denom)
                        tmp.append((float(frameset[row][i+1]) - torsoy)/denom)
                        tmp.append((float(frameset[row][i+2]) - torsoz)/denom)
                        i+=3
                    norm_frameset = np.concatenate((norm_frameset, tmp), axis = 0)
                    
                #norm_frameset = norm_frameset.reshape((k, 42))
               # CiM,iMjoints,iMaxis = initItemMemories (D, MAXL)
                
                predictedLabel = hdcResult(AMM,norm_frameset,CiM,iMjoints,iMaxis,MAXL,compareAngle)
                if(predictedLabel == 1):
                    print("still")
                elif (predictedLabel == 2 ):
                    print ("sit")
                elif (predictedLabel == 3 ):
                    print ("phone")
                k=0
                tmp = []
                frameset = np.array([])
                norm_frameset = np.array([])
        i=i+1
        cv2.waitKey(30)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
          
    

in
sit
in
still
in
phone
in
sit
in
sit
in
phone
in
sit
in
sit
in
still
in
phone
in
sit
in
sit


KeyboardInterrupt: 